In [1]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.impute import SimpleImputer #to handle missing values

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df=pd.read_csv('covid_toy.csv')

In [5]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
51,11,Female,100.0,Strong,Kolkata,Yes
1,27,Male,100.0,Mild,Delhi,Yes
18,64,Female,98.0,Mild,Bangalore,Yes
25,23,Male,NaN,Mild,Mumbai,No
43,22,Female,99.0,Mild,Bangalore,Yes


In [7]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [8]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [9]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

## Without using col transformer(longer way)

In [16]:
from sklearn.impute import SimpleImputer 
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [17]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [19]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [20]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [21]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Using col transform

In [22]:
from sklearn.compose import ColumnTransformer

In [26]:
transform=ColumnTransformer(transformers=[
                            ('tnf1',SimpleImputer(),['fever']),
                            ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
                            ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
                                ],remainder='passthrough')  ##transformers=[] erbhetore tuple pass hoye
                                                           ## inside tuple('transformer name','what to applyi.e transformer object','which col')

In [28]:
transform.fit_transform(X_train).shape

(80, 7)

In [29]:
transform.transform(X_test).shape

(20, 7)